In [5]:
from ansys.workbench.core import launch_workbench

In [15]:
import ansys.fluent.core as pyfluent
from ansys.fluent.core import examples

In [6]:
# launch Workbench service on the local machine; using some options

client_dir = r'D:\users\mvani\PyWB\PyWB-examples\pyfluent_mixing_elbow\client_work_dir'
server_dir = r'D:\users\mvani\PyWB\PyWB-examples\pyfluent_mixing_elbow\server_work_dir'

host = 'localhost'
release = '241'

wb = launch_workbench(release=release, server_workdir=server_dir, client_workdir=client_dir)

In [13]:
wb.upload_file("create_flu_comp.wbjn")

Uploading create_flu_comp.wbjn: 100%|██████████| 87.0/87.0 [00:00<00:00, 9.73kB/s]


In [19]:
wb.upload_file("mixing_elbow.msh.h5")

Uploading mixing_elbow.msh.h5: 100%|██████████| 2.68M/2.68M [00:00<00:00, 399MB/s]


In [14]:
# run a Workbench script to define the project and load geometry

wb.run_script_string('template1 = GetTemplate(TemplateName="FLUENT")', log_level='info')
wb.run_script_string('system1 = template1.CreateSystem()')


{}

###############################################################################
# Launch Fluent
# ~~~~~~~~~~~~~
# Launch Fluent as a service in meshing mode with double precision running on
# two processors.

In [20]:
server_info_file = wb.start_fluent_server(system_name= "FLU")
fluent_session = pyfluent.connect_to_fluent(server_info_filepath= server_info_file)

###############################################################################
# Import mesh and perform mesh check
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Import the mesh and perform a mesh check, which lists the minimum and maximum
# x, y, and z values from the mesh in the default SI units of meters. The mesh
# check also reports a number of other mesh features that are checked. Any errors
# in the mesh are reported. Ensure that the minimum volume is not negative because
# Fluent cannot begin a calculation when this is the case.

In [21]:
fluent_session.file.read(file_type="case", file_name=import_filename)
fluent_session.tui.mesh.check()

Fast-loading "C:\PROGRA~1\ANSYSI~1\v241\fluent\fluent24.1.0\\addons\afd\lib\hdfio.bin"
Done.

AAPBUvD3ch0CsRJ has other FLUENT parallel processes running.                       Process affinity not being set.

Reading from AAPBUvD3ch0CsRJ:"C:\Users\mkvani\AppData\Local\Ansys\ansys_fluent_core\examples\mixing_elbow.msh.h5" in NODE0 mode ...
  Reading mesh ...
       17822 cells,     1 cell zone  ...
          17822 mixed cells,  zone id: 87
       91581 faces,     7 face zones ...
           2168 polygonal wall faces,  zone id: 34
            268 polygonal wall faces,  zone id: 33
            155 polygonal pressure-outlet faces,  zone id: 32
            152 polygonal velocity-inlet faces,  zone id: 31
             55 polygonal velocity-inlet faces,  zone id: 30
           2001 polygonal symmetry faces,  zone id: 29
          86782 mixed interior faces,  zone id: 89
       66417 nodes,     3 node zones ...

Building...
     mesh
     materials,
     interface,
     domains,
     zones,
	

pyfluent.tui WARNING: Currently calling the TUI commands in a generic manner. Please run `python codegen/allapigen.py` from the top-level pyfluent directory to generate the local TUI commands classes.



 Domain Extents:
   x-coordinate: min (m) = -2.000000e-01, max (m) = 2.000000e-01
   y-coordinate: min (m) = -2.250000e-01, max (m) = 2.000000e-01
   z-coordinate: min (m) = 0.000000e+00, max (m) = 4.992265e-02
 Volume statistics:
   minimum volume (m3): 2.443845e-10
   maximum volume (m3): 5.720378e-07
     total volume (m3): 2.500656e-03
 Face area statistics:
   minimum face area (m2): 3.226186e-08
   maximum face area (m2): 7.873458e-05
 Checking mesh............................
Done.

Note: Settings to improve the robustness of pathline and
      particle tracking have been automatically enabled.



###############################################################################
# Set working units for mesh
# ~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set the working units for the mesh to inches. Because the default SI units are
# used for everything except length, you do not have to change any other units
# in this example. If you want working units for length to be other than inches
# (for example, millimeters), make the appropriate change.

In [23]:
fluent_session.tui.define.units("length", "in")

###############################################################################
# Enable heat transfer
# ~~~~~~~~~~~~~~~~~~~~
# Enable heat transfer by activating the energy equation.

In [24]:
fluent_session.setup.models.energy.enabled = True

###############################################################################
# Create material
# ~~~~~~~~~~~~~~~
# Create a material named ``"water-liquid"``.

In [25]:
fluent_session.setup.materials.database.copy_by_name(type="fluid", name="water-liquid")

False

water-liquid copied from database.



###############################################################################
# Set up cell zone conditions
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the cell zone conditions for the fluid zone (``elbow-fluid``). Set ``material``
# to ``"water-liquid"``.

In [26]:
fluent_session.setup.cell_zone_conditions.fluid["elbow-fluid"].material = "water-liquid"

###############################################################################
# Set up boundary conditions for CFD analysis
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Set up the boundary conditions for the inlets, outlet, and walls for CFD
# analysis.

# cold inlet (cold-inlet), Setting: Value:
# Velocity Specification Method: Magnitude, Normal to Boundary
# Velocity Magnitude: 0.4 [m/s]
# Specification Method: Intensity and Hydraulic Diameter
# Turbulent Intensity: 5 [%]
# Hydraulic Diameter: 4 [inch]
# Temperature: 293.15 [K]

In [27]:
cold_inlet = fluent_session.setup.boundary_conditions.velocity_inlet["cold-inlet"]

In [34]:
cold_inlet.get_state()


{'name': 'cold-inlet',
 'momentum': {'velocity_specification_method': 'Magnitude, Normal to Boundary',
  'reference_frame': 'Absolute',
  'velocity': {'option': 'value', 'value': 0},
  'initial_gauge_pressure': {'option': 'value', 'value': 0}},
 'turbulence': {'turbulent_specification': 'Intensity and Viscosity Ratio',
  'turbulent_intensity': 0.05,
  'turbulent_viscosity_ratio_real': 10},
 'thermal': {'t': {'option': 'value', 'value': 300}}}

In [31]:
cold_inlet.vmag.value = 0.4
cold_inlet.ke_spec = "Intensity and Hydraulic Diameter"
cold_inlet.turb_intensity = 0.05
cold_inlet.turb_hydraulic_diam = "4 [in]"
cold_inlet.t = 293.15

AttributeError: vmag is not an allowed Settings objects name.
The most similar names are: thermal.